## Pandas怎样对每个分组应用apply函数?

#### 知识：Pandas的GroupBy遵从split、apply、combine模式

<div style="text-align:left; width:700px;"><img src="./other_files/pandas-split-apply-combine.png" style=""/></div>

这里的split指的是pandas的groupby，我们自己实现apply函数，apply返回的结果由pandas进行combine得到结果

#### GroupBy.apply(function)  
* function的第一个参数是dataframe
* function的返回结果，可是dataframe、series、单个值，甚至和输入dataframe完全没关系

#### 本次实例演示：
1. 怎样对数值列按分组的归一化？
2. 怎样取每个分组的TOPN数据？

### 实例1：怎样对数值列按分组的归一化？

将不同范围的数值列进行归一化，映射到[0,1]区间：
* 更容易做数据横向对比，比如价格字段是几百到几千，增幅字段是0到100
* 机器学习模型学的更快性能更好

归一化的公式：
<div style="text-align:left; width:500px;"><img src="./other_files/Normalization-Formula.jpg" style=""/></div>

#### 演示：用户对电影评分的归一化

每个用户的评分不同，有的乐观派评分高，有的悲观派评分低，按用户做归一化

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv(
    "./datas/movielens-1m/ratings.dat", 
    sep="::",
    engine='python', 
    names="UserID::MovieID::Rating::Timestamp".split("::")
)
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 实现按照用户ID分组，然后对其中一列归一化
def ratings_norm(df):
    """
    @param df：每个用户分组的dataframe
    """
    min_value = df["Rating"].min()
    max_value = df["Rating"].max()
    df["Rating_norm"] = df["Rating"].apply(
        lambda x: (x-min_value)/(max_value-min_value))
    return df

ratings = ratings.groupby("UserID").apply(ratings_norm)

In [4]:
ratings[ratings["UserID"]==1].head()

,UserID,MovieID,Rating,Timestamp,Rating_norm
0,1,1193,5,978300760,1.0
1,1,661,3,978302109,0.0
2,1,914,3,978301968,0.0
3,1,3408,4,978300275,0.5
4,1,2355,5,978824291,1.0


可以看到UserID==1这个用户，Rating==3是他的最低分，是个乐观派，我们归一化到0分；

### 实例2：怎样取每个分组的TOPN数据？

获取2018年每个月温度最高的2天数据

In [5]:
fpath = "./datas/beijing_tianqi/beijing_tianqi_2018.csv"
df = pd.read_csv(fpath)
# 替换掉温度的后缀℃
df.loc[:, "bWendu"] = df["bWendu"].str.replace("℃", "").astype('int32')
df.loc[:, "yWendu"] = df["yWendu"].str.replace("℃", "").astype('int32')
# 新增一列为月份
df['month'] = df['ymd'].str[:7]
df.head()

,ymd,bWendu,yWendu,tianqi,fengxiang,fengli,aqi,aqiInfo,aqiLevel,month
0,2018-01-01,3,-6,晴~多云,东北风,1-2级,59,良,2,2018-01
1,2018-01-02,2,-5,阴~多云,东北风,1-2级,49,优,1,2018-01
2,2018-01-03,2,-5,多云,北风,1-2级,28,优,1,2018-01
3,2018-01-04,0,-8,阴,东北风,1-2级,28,优,1,2018-01
4,2018-01-05,3,-6,多云~晴,西北风,1-2级,50,优,1,2018-01


In [6]:
def getWenduTopN(df, topn):
    """
    这里的df，是每个月份分组group的df
    """
    return df.sort_values(by="bWendu")[["ymd", "bWendu"]][-topn:]

df.groupby("month").apply(getWenduTopN, topn=1).head()

,,ymd,bWendu
month,,,
2018-01,18,2018-01-19,7
2018-02,56,2018-02-26,12
2018-03,85,2018-03-27,27
2018-04,118,2018-04-29,30
2018-05,150,2018-05-31,35


我们看到，grouby的apply函数返回的dataframe，其实和原来的dataframe其实可以完全不一样